## CAI - Colony Avalanche Index - Rebalancing data for January 2024

## Configuring packages

In [20]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyProd
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [21]:
date = '2024-01-01'
min_mcap =  0.1e8
min_weight = 0.005
max_weight = 0.5
circ_supply_threshold = 0.3
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "avalanche-ecosystem"

# Calculating max slippage based on liquidity threshold
liq = 2*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyProd(
                                date,
                                1,
                                "avalanche",
                                "0x48f88A3fE843ccb0b5003e70B4192c1d7448bEf0",
                                "cai_benchmark_data",
                                "cai_liquidities",
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [22]:

results,liquidity = methodology_class.main(
    single_chain="avalanche",
    df_to_remove=[stablecoins,ctokens,atokens],
    ids_to_replace=[["avalanche-2"],["wrapped-avax"]],
    remove_category_assets=['wrapped-bitcoin',"gmx"],
    values_to_update=[{'id':'pangolin','category':'total_supply','value':230000000},
                      {'id':'benqi-liquid-staked-avax','category':'market_cap','value':20000000000},
                      {'id':'benqi-liquid-staked-avax','category':'symbol','value':"sAVAX"}
                      ],
    weight_split_data={"asset_to_split":"wrapped-avax","asset_to_receive":"benqi-liquid-staked-avax","split_ratio":0.6}
    )
liquidity

Note: wrapped-avax, marketcap data available only for 0 < 180 days
Excluding coq-inu, pricing data available only for 28 < 180 days
Note: playa3ull-games-2, marketcap data available only for 163 < 180 days
Note: osaka-protocol, marketcap data available only for 175 < 180 days
Excluding platypus-usd, pricing data available only for 13 < 180 days
Note: domi, marketcap data available only for 31 < 180 days
0xbec243c995409e6520d7c41e404da5deba4b209b avalanche
0x44c784266cf024a60e8acf2427b9857ace194c5d avalanche
0x6b289cceaa8639e3831095d75a3e43520fabf552 avalanche
0x644192291cc835a93d6330b24ea5f5fedd0eef9e avalanche
0xf44ff799ea2bbfec96f9a50498209aac3c2b3b8b avalanche
0x12d8ce035c5de3ce39b1fdd4c1d5a745eaba3b8c avalanche
0xfcaf13227dcbfa2dc2b1928acfca03b85e2d25dd avalanche
Dropping bitcoin-avalanche-bridged-btc-b because a price feed is unavailable
Dropping euro-coin because a price feed is unavailable


,spot price,delivery price,slippage,blockchain
id,,,,
bitcoin-avalanche-bridged-btc-b,2.335000e-05,2.334000e-05,-0.000428,avalanche
euro-coin,9.084970e-01,9.079760e-01,-0.000573,avalanche
benqi-liquid-staked-avax,2.437780e-02,2.432688e-02,-0.002089,avalanche
wrapped-avax,2.738060e-02,2.730037e-02,-0.002930,avalanche
joe,1.527581e+00,1.512440e+00,-0.009912,avalanche
avalaunch,6.840496e-01,6.522075e-01,-0.046549,avalanche
pangolin,1.047097e+01,9.725036e+00,-0.071238,avalanche
benqi,5.355730e+01,4.867408e+01,-0.091177,avalanche
colony,4.148449e+00,3.323376e+00,-0.198887,avalanche


## Show Results

In [23]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
benqi-liquid-staked-avax,sAVAX,2.000000e+10,0.300000,77,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,avalanche
wrapped-avax,AVAX,1.333358e+10,0.200000,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,avalanche
joe,JOE,2.230990e+08,0.406689,104,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,avalanche
avalaunch,XAVA,5.118774e+07,0.093311,23,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4,avalanche


## Contract Output

In [24]:
methodology_class.output_for_contract()

0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,0xd1c3f94de7e5b45fa4edbba472491a9f4b166fc4
77,104,51,23
